In [ ]:
%run vis/xAPISG-noDataToFillVisualization.ipynb # notebook to create the visualisation with a message NoDataToFill
%run vis/helpersFunctions/barCharts.ipynb
%run vis/helpersFunctions/OrderGraphValuesAndSelectDataByXaxis.ipynb

Selectors :
   * xaxisVideo : dropdown value indicating the number of bar per chart (Alls, 5, 25, 50, 10 first, 10 last)
   * orderVideo : dropdown value indicating how to order xlabel : alphabetically or by y value (hight to low or low to hight)
   * percentageVideoSkipped : checkboxe (T/F) value to display the bar chart in percentage or not
   * orderLabelValuesVideo : dropdown value indicating how to order labels : by the sum of the 2 superposed values or by video seen or skipped

In [ ]:
#constructing a dropdown to select how many data we want to see in visualisations for x axis
xaxisVideo=widgets.Dropdown(options=[('All datas',0),('5 per visualisation',5),('25 per visualisation',25),('50 per visualisation',50),('10 first',-1),('10 last',-2)],
                            description='x axe',
                            disabled=False)
#constructing a dropdown to order for alphabetic keys or for values
orderVideo=widgets.Dropdown(options=[('alphabetic',0),('value from highest to lowest',1),('value from lowest to highest',-1)],
                            description='Order',
                            disabled=False)
#constructing a checkboxe to view chart per percentage video skipped
percentageVideoSkipped=widgets.Checkbox(value=False,
                                        description='View per pencentage video skipped')
#constructing a dropdown value indicating how to order labels : by the sum of the 2 superposed values or by video seen or skipped
orderLabelValuesVideo=widgets.Dropdown(options=[("Order by all seen and skipped videos values",0),
                                           ("Order by seens videos values","seen"),
                                           ("Order by skipped videos values","skipped")],
                                        description='Change order label',
                                        disabled=False)

***vis_videos_seen_skipped* function**:
* Output : the widgets selector and the outputVideo in a list to be display in a VBox
* outputVideo : display a bar chart showing for each video the total number of times it has been seen and skipped

In [ ]:
def vis_videos_seen_skipped():
    def display_video_vis(change):
        if orderVideo.value==0:
            orderLabelValuesVideo.layout=layout_hidden
        else:
            orderLabelValuesVideo.layout=layout_visible
        with outputVideo:
            clear_output(wait=False)
            construct_graph_video()
            clear_output(wait=True)
    xaxisVideo.observe(display_video_vis, 'value')
    orderVideo.observe(display_video_vis, 'value')
    percentageVideoSkipped.observe(display_video_vis, 'value')
    orderLabelValuesVideo.observe(display_video_vis, 'value')
    display_video_vis(None)
    return [percentageVideoSkipped, xaxisVideo, orderVideo, orderLabelValuesVideo, outputVideoUpdate, outputVideo]

***construct_graph_video* function**:

Output : Displays bar chart showing for each video the total number of times it has been seen and skipped

In [ ]:
def construct_graph_video():
    #get all players and videos selected
    playersSelected=checkboxesPlayersSelected.get_selected_options()
    videosSelected=checkboxesVideoSelected.get_selected_options()
    if len(videosSelected)>0 and len(playersSelected)>0:
        if xaxisVideo.value in [0,-1,-2]:
            nbgraph=1
        else:
            div=len(videosSelected)/xaxisVideo.value
            nbgraph=math.ceil(div)
        accordionVideo=widgets.Accordion(children=[widgets.Output() for i in range(nbgraph)],
                                         selected_index=None)
        if nbgraph>1:
            for i in range(nbgraph):
                accordionVideo.set_title(i, str(i+1)+"/"+str(nbgraph))
        display(accordionVideo)
        def on_selected_index_video_change(change):
            n=change['new']
            if n!=None:
                #close and clear previous fig of videos
                for fig in figVideoSkipped:
                    plt.close(fig=fig)
                figVideoSkipped.clear()
                #get all players and videos selected
                playersSelected=checkboxesPlayersSelected.get_selected_options()
                videosSelected=checkboxesVideoSelected.get_selected_options()
                #dict with seen and skipped video
                videos_seen = {}
                videos_skipped = {}
                for player in playersSelected:
                    # videos seen
                    for video_seen in players_info[player]["videos_seen"]:
                        if video_seen in videosSelected:
                            if video_seen in videos_seen.keys():
                                videos_seen[video_seen] += 1
                            else:
                                videos_seen[video_seen] = 1
                            if video_seen not in videos_skipped.keys():
                                videos_skipped[video_seen] = 0
                    # videos skipped
                    for video_skipped in players_info[player]["videos_skipped"]:
                        if video_skipped in videosSelected:
                            if video_skipped in videos_skipped.keys():
                                videos_skipped[video_skipped] += 1
                            else:
                                videos_skipped[video_skipped] = 1
                            if video_skipped not in videos_seen.keys():
                                videos_seen[video_skipped] = 0
                            else:
                                videos_seen[video_skipped] -= 1
                #reorganize all in a dictionary
                videosseenskipped={}
                for video in videosSelected:
                    if video in videos_seen.keys():
                        videosseenskipped[video]={"seen":videos_seen[video],"skipped":videos_skipped[video]}
                    else:
                        videosseenskipped[video]={"seen":0,"skipped":0}
                #define order of labels values
                if orderLabelValuesVideo.value==0:
                    lambdafunc=lambda item:item[1]["seen"]+item[1]["skipped"]
                else:
                    lambdafunc=lambda item:item[1][orderLabelValuesVideo.value]
                #order alls values
                videosseenskipped = orderDict(videosseenskipped,orderVideo.value,lambdafunc)
                #separe in sub dict in function of xaxis dropout selected
                nbgraphshown, videosseenskipped,N = organizeDictPerxAxisSelected(videos_seen,xaxisVideo.value, videosseenskipped)
                # Bar chart showing for each video
                # the total number of times it has been seen and skipped
                #get value for video seen and skipped
                v_seen=[videosseenskipped[n][video]["seen"] for video in videosseenskipped[n].keys()]
                v_skipped=[videosseenskipped[n][video]["skipped"] for video in videosseenskipped[n].keys()]
                v_list=[video for video in videosseenskipped[n].keys()]
                #create output for this graph
                with accordionVideo.children[n]:
                    clear_output(wait=False)
                    #create new fig with custom name in the tab
                    name=os.path.splitext(filename)[0]+"_videos_seen_skipped"
                    if percentageVideoSkipped.value:
                        name+="_percentage_skipped"
                    if nbgraphshown>1:
                        name+="_"+str(n+1)+"_"+str(nbgraphshown)
                    fig=plt.figure(num=name, constrained_layout=True)
                    fig.canvas.header_visible = False
                    fig.canvas.layout.min_height = '400px'
                    figVideoSkipped.append(fig)
                    #display graph
                    if percentageVideoSkipped.value:
                        #display percentage bar
                        get_percentage_bar(v_skipped, v_seen, v_list)
                        plt.ylabel("Percentage of videos skipped")
                    else:
                        #display superposed graph
                        if n<nbgraphshown-1:
                            ind = np.arange(N)
                        else:
                            ind= np.arange(len(v_list))
                        get_two_superposed_values_bars(ind,v_list,
                                                        v_seen, "Seen",'orange',
                                                        v_skipped, "Seen but skipped",'lightblue')
                    #orient xticks
                    plt.xticks(rotation=45, ha="right")
                    #add xlabel and title
                    plt.xlabel("Video")
                    plt.title("Videos seen and skipped by players")
                    plt.show()
        accordionVideo.observe(on_selected_index_video_change, 'selected_index')
        accordionVideo.selected_index=0
    else:
        #create new fig with custom name in the tab
        name=os.path.splitext(filename)[0]+"_videos_seen_skipped"
        if percentageVideoSkipped.value:
            name+="_percentage_skipped"
        fig=plt.figure(num=name, constrained_layout=True)
        fig.canvas.header_visible = False
        fig.canvas.layout.min_height = '400px'
        figVideoSkipped.append(fig)
        noDataToFillVis(10)
        #orient xticks
        plt.xticks(rotation=45, ha="right")
        #add xlabel and title
        plt.xlabel("Video")
        if percentageVideoSkipped.value:
            plt.ylabel("Percentage of videos skipped")
        else:
            plt.ylabel("Number of times")
        plt.title("Videos seen and skipped by players")
        plt.show()
    with outputVideoUpdate:
        clear_output(wait=False)